In [2]:
import polars as pl
import numpy as np
import altair as alt
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [3]:
df = pl.read_parquet('../data/processed/cg_context_test.parquet')
df.head()

read_name,cg_pos,window_seq,window_fi,window_fp,window_ri,window_rp
str,i64,str,list[i64],list[i64],list[i64],list[i64]
"""m64168_200823_191315/30/ccs""",493,"""TGTTAAGGGCAGCTAGAGAGAAAGGCCAGG…","[19, 22, … 31]","[11, 28, … 27]","[25, 15, … 16]","[10, 19, … 14]"
"""m64168_200823_191315/30/ccs""",1164,"""AGGGATTTTAGGCCTATCTCTTAACCTTTA…","[10, 44, … 20]","[17, 17, … 29]","[17, 19, … 18]","[16, 20, … 15]"
"""m64168_200823_191315/30/ccs""",1786,"""TTCATGATGATGGTGGAGGCACAAGATAGT…","[24, 17, … 14]","[18, 22, … 15]","[17, 19, … 43]","[14, 21, … 26]"
"""m64168_200823_191315/30/ccs""",1810,"""GATAGTGCGATTGTATCCCTGAATAACCTT…","[30, 32, … 40]","[19, 23, … 17]","[13, 47, … 20]","[23, 13, … 23]"
"""m64168_200823_191315/30/ccs""",2042,"""ATACATATATAATTGTCAATCATTAAAACA…","[32, 18, … 27]","[12, 11, … 14]","[50, 15, … 12]","[14, 14, … 16]"


In [4]:
ipd_fwd_array_all = np.stack([s.to_numpy() for s in df['window_fi']])
ipd_rev_array_all = np.stack([s.to_numpy() for s in df['window_ri']])

df_long_fwd=pl.DataFrame(ipd_fwd_array_all, schema = [f'{i}' for i in range(64)]).unpivot(variable_name='index', value_name='ipd').cast({"index": pl.Int64})
df_long_fwd = df_long_fwd.with_columns(strand = pl.lit('fwd'))
df_long_rev=pl.DataFrame(ipd_rev_array_all, schema = [f'{i}' for i in range(64)]).unpivot(variable_name='index', value_name='ipd').cast({"index": pl.Int64})
df_long_rev = df_long_rev.with_columns(strand = pl.lit('rev'))

df_long = pl.concat([df_long_fwd, df_long_rev], how='vertical_relaxed')
df_long.shape


(10630912, 3)

In [5]:
rev_fwd_ipd_at_index = alt.Chart(df_long).mark_line(point=True,).encode(
    alt.X('index:O'),
    alt.Y('mean(ipd):Q', title = 'index-Wise IPD Mean').scale(domain=(20,30), clamp=True),
    alt.Color('strand:N', legend=alt.Legend(title="fwd-rev strand"))
).properties(
    title = 'IPD Means at Indices Local to Observed Mutations',
    width = 800,
    height = 600
)
rev_fwd_ipd_at_index_ci = alt.Chart(df_long).mark_errorbar(extent='ci').encode(
    alt.X('index:O'),
    alt.Y('ipd:Q', title='95% Bootstrapped CI'),
    alt.Color('strand:N', legend=alt.Legend(title="fwd-rev strand"))
)

rev_fwd_ipd_64 = rev_fwd_ipd_at_index + rev_fwd_ipd_at_index_ci

In [ ]:
rev_fwd_ipd_64.save('../plots/index_kinetics.svg')